In [1]:
#Initialization and get all file names under a directory
import os
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import pandas as pd
import yaml
_join = os.path.join

CONFIG = r'T:/ABM/ABM_FY21/Sketch_Planning/VisionEval/Model/Update_Automations/settings.yml'
with open(CONFIG) as cff:
    config =yaml.safe_load(cff)
target_path = ''.join(config['target_path'])
scenpath = ''.join(config['scenpath'])
scenario_list = config['scenario_list']
year_List = [x[0] for x in scenario_list]

mypath = os.path.normpath(os.path.join(target_path, *([".."] * 1), r'VERSPM'))
onlyfiles = [f for f in listdir(join(mypath, 'inputs')) if isfile(join(mypath, 'inputs',f))]

In [2]:
# update geospatial information for all input files
df_template = pd.read_csv(join(target_path, 'defs', 'geo.csv'))

#update geospatial information for all input files

In [3]:
# Azone
new_List = df_template['Azone'].unique().tolist()
new_List.sort()
azone_List = [new_List[i//2] for i in range(len(new_List)*len(year_List))]
year_npList = year_List * len(new_List)

In [4]:
# Bzone
b_listYear = year_List * len(df_template)
b_listGeo = df_template['Bzone'].tolist() * len(year_List)
b_listYear.sort()

In [5]:
# Marea
m_list = df_template['Marea'].unique().tolist()
m_list = [x for x in m_list if pd.isnull(x) == False]
try:
    m_list.remove('None')
except:
    pass
m_listYear = year_List * len(m_list)
m_list_final = m_list * len(year_List)
m_list_final.sort()
m_list_final2 = list(set(m_list_final))

In [7]:
for file in onlyfiles:
    df_file = pd.read_csv(join(mypath, 'inputs', file))
    head_List = list(df_file.columns.values)
    df_new = pd.DataFrame({}, columns = head_List, index=None)    
    if 'azone' in file:
        df_new['Geo'] = azone_List 
        df_new['Year'] = year_npList  
        df_new = df_new.sort_values(["Year", "Geo"], ascending = (True, True))
        df_new.to_csv(join(target_path, 'inputs', file),index=False,)
    elif 'bzone' in file:
        df_new['Geo'] = b_listGeo
        df_new['Year'] = b_listYear
        df_new.to_csv(join(target_path, 'inputs', file),index=False,)
    elif 'region' in file:
        if 'base' not in file:
            df_new['Year'] = year_List
        df_new.to_csv(join(target_path, 'inputs', file),index=False,) 
    elif 'marea' in file:
        # so far, the Geo column is blank. Wait until when marea is defined.
        if 'Year' in list(df_new):
            df_new['Year'] = m_listYear
            df_new['Geo'] = m_list_final    
        else:
            df_new['Geo'] = m_list_final2   
        df_new.to_csv(join(target_path, 'inputs', file),index=False,)   